<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/Text_Summarization_with_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install bs4

In [54]:
    import numpy as np
    import pandas as pd
    import re
    import matplotlib.pyplot as plt
    from nltk.corpus import stopwords
    from bs4 import BeautifulSoup
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import urllib.request
    np.random.seed(seed=0)

In [55]:
data=pd.read_csv('Reviews.csv')
print('전체 리뷰의 개수 : ', len(data))

전체 리뷰의 개수 :  568454


In [56]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [57]:
data=data[['Text', 'Summary']]

In [58]:
data.sample(10)

,Text,Summary
11214,I could live off of these! I'm not kidding. Th...,Yes!!!
222079,I found this product in a local grocery store ...,Tropical Trail Mix
251082,My young son is taking Chinese in school and w...,Great Gummies!
317731,I love Grove Square Spiced Apple Cider Cups (s...,I'm addicted to this!
404652,This is probably my shortest review but there'...,"Tastes Eh, Very Expensive"
483320,This is a great instant ginger product. I mix...,Very Good
435467,I have another pumpkin eating dog (it really h...,Pleasant Surprise
322422,To my knowledge Late july has been one of the ...,"Great for Students, but not the best organic c..."
496171,If you love nuts and want something truly natu...,My KIND of bar
427084,(This review is for the white can with the gol...,Like Battery Acid


In [59]:
print('Text 열에서 중복을 배제한 유일한 샘플의 수 : ', data['Text'].nunique())
print('Summary 열에서 중복을 배제한 유일한 샘플의 수 : ', data['Summary'].nunique())

Text 열에서 중복을 배제한 유일한 샘플의 수 :  393579
Summary 열에서 중복을 배제한 유일한 샘플의 수 :  295742


In [60]:
data.isnull().sum()

Text        0
Summary    27
dtype: int64

In [61]:
data.dropna(axis=0, inplace=True)

In [62]:
data.drop_duplicates(subset=['Text'], inplace=True)
print('전체 샘플의 수 : ', len(data))

전체 샘플의 수 :  393576


In [63]:
# 전처리 함수 내 사용
contractions = {"'cause": 'because',
 "I'd": 'I would',
 "I'd've": 'I would have',
 "I'll": 'I will',
 "I'll've": 'I will have',
 "I'm": 'I am',
 "I've": 'I have',
 "ain't": 'is not',
 "aren't": 'are not',
 "can't": 'cannot',
 "could've": 'could have',
 "couldn't": 'could not',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he would',
 "he'll": 'he will',
 "he's": 'he is',
 "here's": 'here is',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how is',
 "i'd": 'i would',
 "i'd've": 'i would have',
 "i'll": 'i will',
 "i'll've": 'i will have',
 "i'm": 'i am',
 "i've": 'i have',
 "isn't": 'is not',
 "it'd": 'it would',
 "it'd've": 'it would have',
 "it'll": 'it will',
 "it'll've": 'it will have',
 "it's": 'it is',
 "let's": 'let us',
 "ma'am": 'madam',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "o'clock": 'of the clock',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "sha'n't": 'shall not',
 "shan't": 'shall not',
 "shan't've": 'shall not have',
 "she'd": 'she would',
 "she'd've": 'she would have',
 "she'll": 'she will',
 "she'll've": 'she will have',
 "she's": 'she is',
 "should've": 'should have',
 "shouldn't": 'should not',
 "shouldn't've": 'should not have',
 "so's": 'so as',
 "so've": 'so have',
 "that'd": 'that would',
 "that'd've": 'that would have',
 "that's": 'that is',
 "there'd": 'there would',
 "there'd've": 'there would have',
 "there's": 'there is',
 "they'd": 'they would',
 "they'd've": 'they would have',
 "they'll": 'they will',
 "they'll've": 'they will have',
 "they're": 'they are',
 "they've": 'they have',
 "this's": 'this is',
 "to've": 'to have',
 "wasn't": 'was not',
 "we'd": 'we would',
 "we'd've": 'we would have',
 "we'll": 'we will',
 "we'll've": 'we will have',
 "we're": 'we are',
 "we've": 'we have',
 "weren't": 'were not',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "what're": 'what are',
 "what's": 'what is',
 "what've": 'what have',
 "when's": 'when is',
 "when've": 'when have',
 "where'd": 'where did',
 "where's": 'where is',
 "where've": 'where have',
 "who'll": 'who will',
 "who'll've": 'who will have',
 "who's": 'who is',
 "who've": 'who have',
 "why's": 'why is',
 "why've": 'why have',
 "will've": 'will have',
 "won't": 'will not',
 "won't've": 'will not have',
 "would've": 'would have',
 "wouldn't": 'would not',
 "wouldn't've": 'would not have',
 "y'all": 'you all',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "you'd": 'you would',
 "you'd've": 'you would have',
 "you'll": 'you will',
 "you'll've": 'you will have',
 "you're": 'you are',
 "you've": 'you have'}

In [64]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
#nltk 불용어
stop_words=set(stopwords.words('english'))
print('불용어의 개수 : ', len(stop_words))

불용어의 개수 :  179


In [66]:
# 전처리 함수
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","",sentence) # 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah

    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens


In [67]:
temp_text='Everything I bought was great, infact I ordered twice and the third orderd was<br \>for my mother and father.'
temp_summary='Great way to start (or finish) the day!!!'
print(preprocess_sentence(temp_text))
print(preprocess_sentence(temp_summary, 0))

everything bought great infact ordered twice third orderd wasfor mother father
great way to start the day


In [ ]:
#Text 열 전처리
clean_text=[]
for s in data['Text']:
  clean_text.append(preprocess_sentence(s))
clean_text[:5]

<ipython-input-66-486d11b728f3>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거


In [ ]:
clean_summary=[]
for s in data['Summary']:
  clean_summary.append(preprocess_sentence(s))
clean_summary[:5]

In [ ]:
data['Text']=clean_text
data['Summary']=clean_summary

In [ ]:
data.replace('', np.nan, inplace=True)
print(data.isnull().sum())

In [ ]:
data.dropna(axis=0, inplace=True)
print('전체 샘플 수 : ', len(data))

In [ ]:
#길이 분포
text_len=[len(s.split())for s in data['Text']]
summary_len=[len(s.split())for s in data['Summary']]

print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
print('요약의 최소 길이 : {}'.format(np.min(len(summary_len))))
print('요약의 최대 길이 : {}'.format(np.max(len(summary_len))))
print('요약의 평균 길이 : {}'.format(np.mean(len(summary_len))))

plt.subplot(1,2,1)
plt.boxplot(summary_len)
plt.title('Summary')
plt.subplot(1,2,2)
plt.boxplot(text_len)
plt.title('Text')
plt.tight_layout()
plt.show()

plt.title('Summary')
plt.hist(summary_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

plt.title('Text')
plt.hist(text_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
text_max_len=50
summary_max_len=8

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt=0
  for s in nested_list:
    if(len(s.split())<=max_len):
      cnt+=1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율 : %s'%(max_len, (cnt/len(nested_list))))

In [ ]:
below_threshold_len(text_max_len, data['Text'])

In [ ]:
below_threshold_len(summary_max_len, data['Summary'])

In [ ]:
data = data[data['Text'].apply(lambda x: len(x.split()) <= text_max_len)]
data = data[data['Summary'].apply(lambda x: len(x.split()) <= summary_max_len)]
print('전체 샘플수 :',(len(data)))

In [ ]:
data['decoder_input']=data['Summary'].apply(lambda x:'sostoken ' + x)
data['decoder_target']=data['Summary'].apply(lambda x:x + ' eostoken')
data.head()

In [ ]:
encoder_input=np.array(data['Text'])
decoder_input=np.array(data['decoder_input'])
decoder_target=np.array(data['decoder_target'])

In [ ]:
indices=np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

In [ ]:
encoder_input=encoder_input[indices]
decoder_input=decoder_input[indices]
decoder_target=decoder_target[indices]

In [ ]:
n_of_val=int(len(encoder_input)*0.2)
print('테스트 데이터의 수 : ', n_of_val)

In [ ]:
encoder_input_train=encoder_input[:-n_of_val]
decoder_input_train=decoder_input[:-n_of_val]
decoder_target_train=decoder_target[:-n_of_val]

encoder_input_test=encoder_input[-n_of_val:]
decoder_input_test=decoder_input[-n_of_val:]
decoder_target_test=decoder_target[-n_of_val:]

In [ ]:
print('훈련 데이터의 개수 : ', len(encoder_input_train))
print('훈련 레이블의 개수 : ', len(decoder_input_train))
print('테스트 데이터의 개수 : ', len(encoder_input_test))
print('테스트 레이블의 개수 : ', len(decoder_input_test))

In [ ]:
src_tokenizer=Tokenizer()
src_tokenizer.fit_on_texts(encoder_input_train)

In [ ]:
#등장 빈도수 제한
threshold=7
total_cnt=len(src_tokenizer.word_index)#단어의 수
rare_cnt=0
total_freq=0
rare_freq=0

#단어와 빈도수
for key, value in src_tokenizer.word_counts.items():
  total_freq+=value

  #단어의 등장 빈도수가 threshold보다 적으면
  if(value<threshold):
    rare_cnt+=1
    rare_freq+=value

print('단어 집합의 크기 : ', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수 : %s'%(threshold-1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 : ', total_cnt-rare_cnt)
print('단어 집합에서 희귀 단어의 비율 : ', (rare_cnt/total_cnt)*100)
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ', (rare_freq/total_freq)*100)

In [ ]:
src_vocab=17000
src_tokenizer=Tokenizer(num_words=src_vocab)
src_tokenizer.fit_on_texts(encoder_input_train)

#텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train=src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test=src_tokenizer.texts_to_sequences(encoder_input_test)

In [ ]:
print(encoder_input_train[:3])

In [ ]:
tar_tokenizer=Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)

In [ ]:
threshold=6
total_cnt=len(tar_tokenizer.word_index)
rare_cnt=0
total_freq=0
rare_freq=0

for key, value in tar_tokenizer.word_counts.items():
  total_freq+=value
  if(value<threshold):
    rare_cnt+=1
    rare_freq+=value

print('단어 집합의 크기 : ', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수 : %s'%(threshold-1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 : ', total_cnt-rare_cnt)
print('단어 집합에서 희귀 단어의 비율 : ', (rare_cnt/total_cnt)*100)
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ', (rare_freq/total_freq)*100)

In [ ]:
tar_vocab=7000
tar_tokenizer=Tokenizer(num_words=tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

In [ ]:
decoder_input_train=tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_input_test=tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_train=tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_target_test=tar_tokenizer.texts_to_sequences(decoder_target_test)

In [ ]:
print(decoder_input_train[:5])

In [ ]:
print(decoder_target_train[:5])

In [ ]:
#eostoken과 sostoken을 포함하여 길이가 1인 summary의 인덱스를 저장
drop_train=[index for index, sentence in enumerate(decoder_input_train) if len(sentence)==1]
drop_test=[index for index, sentence in enumerate(decoder_input_test) if len(sentence)==1]

In [ ]:
print('삭제할 훈련의 수 : ', len(drop_train))
print('삭제할 테스트의 수 : ', len(drop_test))

In [ ]:
encoder_input_train=np.delete(encoder_input_train, drop_train, axis=0)
decoder_input_train=np.delete(decoder_input_train, drop_train, axis=0)
decoder_target_train=np.delete(decoder_target_train, drop_train, axis=0)

encoder_input_test=np.delete(encoder_input_test, drop_test, axis=0)
decoder_input_test=np.delete(decoder_input_test, drop_test, axis=0)
decoder_target_test=np.delete(decoder_target_test, drop_test, axis=0)

print('훈련 데이터의 개수 : ', len(encoder_input_train))
print('훈련 레이블의 개수 : ', len(decoder_input_train))
print('테스트 데이터의 개수 : ', len(encoder_input_test))
print('테스트 레이블의 개수 : ', len(decoder_input_test))

In [ ]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen = text_max_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen = text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen = summary_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen = summary_max_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen = summary_max_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen = summary_max_len, padding='post')


In [ ]:
#seq2seq+attention으로 요약 모델 설계 및 훈련
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
embedding_dim=128
hidden_size=256

#인코더
encoder_inputs=Input(shape=(text_max_len,))

#인코드의 임베딩 층
enc_emb=Embedding(src_vocab, embedding_dim)(encoder_inputs)

#인코더의 LSTM1
encoder_lstm1=LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1=encoder_lstm1(enc_emb)

#인코더의 LSTM2
encoder_lstm2=LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2=encoder_lstm2(encoder_output1)

#인코더의 LSTM3
encoder_lstm3=LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output3, state_h, state_c=encoder_lstm3(encoder_output2)

In [ ]:
#디코더
decoder_inputs=Input(shape=(None,))

#디코더의 임베딩 층
dec_emb_layer=Embedding(tar_vocab, embedding_dim)
dec_emb=dec_emb_layer(decoder_inputs)

#디코더의 LSTM
decoder_lstm=LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
decoder_outputs, _, _=decoder_lstm(dec_emb, initial_state=[state_h, state_c])

In [ ]:
#디코더의 출력층
decoder_softmax_layer=Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs=decoder_softmax_layer(decoder_outputs)

#모델 정의
model=Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

In [ ]:
#attention함수 사용
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/20.%20Text%20Summarization%20with%20Attention/attention.py", filename="attention.py")
from attention import AttentionLayer

In [ ]:
#어텐션 층
attn_layer=AttentionLayer(name='attention_layer')
attn_out,attn_states=attn_layer([encoder_outpus, decoder_outputs])
#어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input=Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#디코더의 출력층
decoder_softmax_layer=Dens(tar_vocab, activation='softmax')
decoder_softmax_outputs=decoder_softmax_layer(decoder_concat_input)

#모델 정의
model=Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
history=model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), batch_size=256, callbacks=[es], epochs=50)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
src_index_to_word=src_tokenizer.index_word
tar_word_to_index=tar_tokenizer.word_index
tar_index_to_word=tar_tokenizer.index_word

In [ ]:
#seq2seq은 훈련단계와 테스트 단계가 다르므로 모델을 별도로 다시 설계한다.
encoder_model=Model(inputs=encoder_inputs, outputs=[decoder_outputs, state_h, state_c])

In [ ]:
#이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h=Input(shape=(hidden_size,))
decoder_state_input_c=Input(shape=(hidden_size,))

dec_emb2=dec_emb_layer(decoder_inputs)
#문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)을 이전 시점의 상태로 사용, 이는 뒤의 함수 decode_sequence()에 구현
#훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태과 셀 상태인 state_h와 state_c를 버리지 않음
decoder_outputs2, state_h2, state_c2=decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

In [ ]:
#어텐션 함수
decoder_hidden_state_input=Input(shape=(text_max_len, hidden_size))
attn_out_inf, attn_states_inf=attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat=Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

#디코더의 출력층
decoder_outputs2=decoder_softmax_layer(decoder_inf_concat)

#최종 디코더 모델
decoder_model=Model([decoder_inputs]+[decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c], [decoder_outputs2]+[state_h2, state_c2])